In [ ]:
!pip install --upgrade progressbar2

  Attempting uninstall: progressbar2
    Found existing installation: progressbar2 3.38.0
    Uninstalling progressbar2-3.38.0:
      Successfully uninstalled progressbar2-3.38.0


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 636 kB 32.4 MB/s 
     |████████████████████████████████| 895 kB 36.4 MB/s 
     |████████████████████████████████| 3.3 MB 35.2 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# Define root directory from Google Drive
from google.colab import drive
import os
drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)
os.chdir("/content/drive/My Drive/trustpilot")
os.listdir()

Mounted at /content/drive
Mounted at /content/drive


['united_states.auto-adjusted_gender.geocoded.json',
 'france.auto-adjusted_gender.NUTS-regions.json',
 'france.json',
 'trustpilot.pt',
 '.ipynb_checkpoints',
 'united_states.json',
 'trustpilot_france.pt',
 'trustpilot_usa.pt',
 'trustpilot_usa_bert.pt']

In [ ]:
# Import Torch
import torch
# Setup device
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(dev)

cuda


In [ ]:
import json
  
# Opening JSON file
f = open('united_states.json',)
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Closing file
f.close()

In [ ]:
usa_reviews = []
usa_targets = []

In [ ]:
from string import punctuation
print(len(data))
count = 0
for i in range(0, len(data)):
  if (data[i]['reviews'][0]['text'] != [] and (''.join([c for c in data[i]['reviews'][0]['text'][0] if c not in punctuation]))!= '' and count < 30000):
    usa_reviews.append(data[i]['reviews'][0]['text'][0])
    usa_targets.append(int(data[i]['reviews'][0]['rating']))
    count = count + 1

648784


In [ ]:
print(len(usa_reviews))
print(len(usa_targets))

30000
30000


In [ ]:
count=0
for r in range(0, len(usa_reviews)):
  if len(usa_reviews[r])<= 0:
    print(r)
    count=count+1

print(count)

0


In [ ]:
# Load sentiments: less than 3 stars is negative (0), 4 and 5 are positive (1)   
sentiments = [0 if f <= 3 else 1 for f in usa_targets]

print(len(sentiments))

30000


In [ ]:
print(sentiments[10])

1


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(usa_reviews, sentiments, test_size=0.33)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33)

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
                                      'bert-base-uncased', 
                                      num_labels = 2,
                                      output_attentions = False,
                                      output_hidden_states = False
                                     )

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
import transformers
import torch.nn as nn

class BERTClassification(nn.Module):
    def __init__ (self):
        super(BERTClassification, self).__init__()
        self.bert = transformers.BertModel.from_pretrained( 'bert-base-uncased', num_labels = 2, output_attentions = False, output_hidden_states = False)
        self.bert_drop = nn.Dropout(0.4)
        self.out = nn.Linear(768, 2)
        
    def forward(self, input_ids, attention_mask, token_type_ids):
        _, pooledOut = self.bert(input_ids, attention_mask = attention_mask,
                                token_type_ids=token_type_ids)
        bertOut = self.bert_drop(pooledOut)
        output = self.out(bertOut)
        
        return output

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    X_train,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
encoded_data_val = tokenizer.batch_encode_plus(
    X_val,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
encoded_data_test = tokenizer.batch_encode_plus(
    X_test,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(y_train)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(y_val)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(y_test)

In [ ]:
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val, 
                            attention_masks_val,
                           labels_val)

dataset_test = TensorDataset(input_ids_test, 
                            attention_masks_test,
                           labels_test)

In [ ]:
print(len(dataset_train))
print(len(dataset_val))
print(len(dataset_test))
print(len(dataset_train)+len(dataset_val)+len(dataset_test))

13467
6633
9900
30000


In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
#model = BERTClassification()

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 4

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

dataloader_test = DataLoader(
    dataset_test,
    sampler=RandomSampler(dataset_test),
    batch_size=32
)

In [ ]:
loaders = {"train": dataloader_train,
           "val":   dataloader_val,
           "test":  dataloader_test}

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

In [ ]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')


def accuracy_per_class(preds, labels):
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')



In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)


cuda


In [ ]:
import progressbar as pb
# Start training
for epoch in range(150):
    # Initialize accumulators for computing average loss/accuracy
    epoch_loss_sum = {'train': 0, 'val': 0, 'test': 0}
    epoch_loss_cnt = {'train': 0, 'val': 0, 'test': 0}
    epoch_accuracy_sum = {'train': 0, 'val': 0, 'test': 0}
    epoch_accuracy_cnt = {'train': 0, 'val': 0, 'test': 0}
    # Process each split
    for split in ["train", "val", "test"]:
        # Set network mode
        if split == "train":
            model.train()
            torch.set_grad_enabled(True)
            widgets = [
              ' [', pb.Timer(), '] ',
              pb.Bar(),
              ' [', pb.ETA(), '] ', pb.Variable('ta','[Train Acc: {formatted_value}]')
            ]

            progbar = pb.ProgressBar(max_value=len(loaders[split]),widgets=widgets,redirect_stdout=True)

        else:
            model.eval()
            torch.set_grad_enabled(False)
        # Process all data in split
        for batch in loaders[split]:
          model.zero_grad()
          batch = tuple(b.to(device) for b in batch)
          inputs = {
              'input_ids': batch[0],
              'attention_mask': batch[1],
              'labels': batch[2]
          }
          
          output = model(**inputs)
          loss = output.loss
          #print(output)
          epoch_loss_sum[split] +=loss.item()
          epoch_loss_cnt[split] += 1
          
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
          
          optimizer.step()
          scheduler.step()

          logits = output[1]

          logits = logits.detach().cpu().numpy()
          label_ids = batch[2].cpu().numpy()
          pred = logits
          pred = np.argmax(pred, axis=1).flatten()
          label_ids = label_ids.flatten()
          correct = 0
          for i in range(0, len(label_ids)):
            if pred[i] == label_ids[i]:
              correct = correct + 1
          accuracy = correct/batch[2].size(0)
          
          # Update accuracy sum
          epoch_accuracy_sum[split] += accuracy
          epoch_accuracy_cnt[split] += 1
          if (split=='train'):
            progbar.update(i, ta=accuracy)
          # Backward and optimize
          if split == "train":
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()
              scheduler.step()

    # Compute average epoch loss/accuracy
    avg_train_loss = epoch_loss_sum["train"]/epoch_loss_cnt["train"]
    avg_train_accuracy = epoch_accuracy_sum["train"]/epoch_accuracy_cnt["train"]
    avg_val_loss = epoch_loss_sum["val"]/epoch_loss_cnt["val"]
    avg_val_accuracy = epoch_accuracy_sum["val"]/epoch_accuracy_cnt["val"]
    avg_test_loss = epoch_loss_sum["test"]/epoch_loss_cnt["test"]
    avg_test_accuracy = epoch_accuracy_sum["test"]/epoch_accuracy_cnt["test"]
    if (progbar is not None):
          progbar.finish()
    print(f"Epoch: {epoch+1}, TrL={avg_train_loss:.4f}, TrA={avg_train_accuracy:.4f},",
                            f"VL={avg_val_loss:.4f}, VA={avg_val_accuracy:.4f}, ",
                            f"TeL={avg_test_loss:.4f}, TeA={avg_test_accuracy:.4f}")

 [Elapsed Time: 0:14:57] |      | [ETA:  11 days, 15:28:35] [Train Acc:    1.0]

KeyboardInterrupt: ignored

In [ ]:
progbar.finish()

 [Elapsed Time: 0:15:07] |###############| [Time:  0:15:07] [Train Acc:    1.0]


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
from tqdm.notebook import tqdm

for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
    
    #torch.save(model.state_dict(), f'Models/BERT_ft_Epoch{epoch}.model')
    
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/3367 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.13173457719785142


  0%|          | 0/208 [00:00<?, ?it/s]

Validation loss: 0.17692985197209055
F1 Score (weighted): 0.9586657387193604


...


Epoch {epoch}
Training loss: 0.01819909354720155


  0%|          | 0/208 [00:00<?, ?it/s]

Validation loss: 0.36201198488228886
F1 Score (weighted): 0.9551546083369455


Epoch 6:   0%|          | 0/3367 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: 0
Accuracy:435/613

Class: 1
Accuracy:5908/6020



In [ ]:
# Test review
#review = "good quality of the service"
review = "the product does not work worst service ever"

encoded_data = tokenizer.batch_encode_plus(
    review,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)
input_ids = encoded_data['input_ids']
attention_masks = encoded_data['attention_mask']
labels = torch.tensor(0)
inputs = {
            'input_ids': input_ids,
            'attention_mask': attention_masks,
            'labels': labels
        }
model.eval() 
model.cpu()       
outputs = model(**inputs)
loss = outputs[0]
logits = outputs[1]

logits = logits.detach().cpu().numpy()
pred = logits



print(pred)

In [ ]:
#save model
model_save_name = 'trustpilot_usa_bert.pt'
path = F"/content/drive/My Drive/trustpilot/{model_save_name}" 
torch.save(model.state_dict(), path)

In [ ]:
#load model
model_name = 'trustpilot_usa_bert.pt'
path = F"/content/drive/My Drive/trustpilot/{model_name}" 
model.load_state_dict(torch.load(path))
